# Fit pure component molecular parameters for Hexane

This notebook has te purpose of showing how to optimize the molecular parameters of a pure fluid in SGTPy.

First it's needed to import the necessary modules

In [1]:
import numpy as np
from scipy.optimize import minimize
from sgtpy import component, saftvrmie

Now the experimental equilibria data is read. For Hexane this include vapor and liquid density and saturation temperature and pressure.

In [2]:
# Experimental equilibria data obtained from NIST WebBook
Tsat = np.array([290., 300., 310., 320., 330., 340., 350., 360.]) # K
Psat = np.array([ 14016.,  21865.,  32975.,  48251.,  68721.,  95527., 129920., 173260.]) # Pa
rhol = np.array([7683.6, 7577.4, 7469.6, 7360.1, 7248.7, 7135. , 7018.7, 6899.5]) #nol/m3
rhov = np.array([ 5.8845,  8.9152, 13.087, 18.683, 26.023, 35.466, 47.412, 62.314]) #mol/m3

Then is necessary to create an objective function, as ```fobj```. This function can be modified according to the available experimental data and the parameters to be optimized. 

For this fluid, $m_s, \sigma, \epsilon, \lambda_r$ are optimized and $\lambda_a$ is fixed to 6. The objective function measures the error for the calculated saturation pressure, liquid density and vapor density (weighted). 

In [3]:
# objective function to optimize molecular parameters
def fobj(inc):
    ms, sigma, eps, lambda_r = inc
    pure = component(ms = ms, sigma = sigma , eps = eps, lambda_r = lambda_r , lambda_a = 6.)
    eos = saftvrmie(pure)
    
    #Pure component pressure and liquid density
    P = np.zeros_like(Psat) 
    vl = np.zeros_like(rhol)
    vv = np.zeros_like(rhov)
    n= len(Psat)
    for i in range(n):
        P[i], vl[i], vv[i] = eos.psat(Tsat[i], Psat[i])
    
    rhosaftl = 1/vl
    rhosaftv = 1/vv
    
    error = np.mean(np.abs(P/Psat - 1))
    error += np.mean(np.abs(rhosaftl/rhol - 1))
    error += 0.1*np.mean(np.abs(rhosaftv/rhov - 1))
    
    return error

The objective function is minimized using SciPy's ```minimize``` function.

In [4]:
# initial guess for ms, sigma, eps and lambda_r
inc0 = np.array([2.0, 4.52313581 , 378.98125026,  19.00195008])
minimize(fobj, inc0, method = 'Nelder-Mead')

c:\users\gusta\documents\sgtpy\sgtpy\vrmie_pure\ideal.py:25: RuntimeWarning: invalid value encountered in log
  a = np.log(rho * broglie_vol**3) - 1
c:\users\gusta\documents\sgtpy\sgtpy\vrmie_pure\saftvrmie.py:280: RuntimeWarning: overflow encountered in exp
  integrer = np.exp(-beta * self.umie)
c:\users\gusta\documents\sgtpy\sgtpy\vrmie_pure\monomer_aux.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  I = (x0**lam3 - 1.) / lam3
c:\users\gusta\documents\sgtpy\sgtpy\vrmie_pure\monomer_aux.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  J = (lam3*x0**lam4 - lam4*x0**lam3 + 1.)/(lam3 * lam4)
c:\users\gusta\documents\sgtpy\sgtpy\vrmie_pure\monomer_aux.py:15: RuntimeWarning: invalid value encountered in double_scalars
  J = (lam3*x0**lam4 - lam4*x0**lam3 + 1.)/(lam3 * lam4)
c:\users\gusta\documents\sgtpy\sgtpy\vrmie_pure\saftvrmie.py:401: RuntimeWarning: divide by zero encountered in log
  Kab = np.log((rc + 2*rd)/dia)
c:\users\gusta\documents\sgtp

 final_simplex: (array([[  1.96834513,   4.54625563, 376.94035316,  18.34400627],
       [  1.96834502,   4.54625559, 376.94031409,  18.34399954],
       [  1.96834441,   4.54625617, 376.94044338,  18.34400805],
       [  1.96834458,   4.54625598, 376.94039329,  18.34400432],
       [  1.96834461,   4.54625598, 376.94033416,  18.34399723]]), array([0.00188235, 0.00188236, 0.00188236, 0.00188236, 0.00188236]))
           fun: 0.0018823536739056624
       message: 'Optimization terminated successfully.'
          nfev: 233
           nit: 128
        status: 0
       success: True
             x: array([  1.96834513,   4.54625563, 376.94035316,  18.34400627])